<a href="https://colab.research.google.com/github/vdubya/criteria-assistant/blob/main/UFGS_CheckAllURLs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install lxml pandas openpyxl httpx[http2] zipfile36

import zipfile
import os
import re
import pandas as pd
import httpx
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures # For as_completed
import time
import shutil # For file operations like checking existence and removing
import sys # Import sys to use sys.exit() reliably
# Imports needed for Excel formatting
from openpyxl.styles import Font, PatternFill, colors, Alignment, Border, Side, NumberFormatDescriptor
from openpyxl.formatting.rule import FormulaRule
from openpyxl.utils import get_column_letter # To convert column index to letter
from openpyxl.worksheet.worksheet import Worksheet # For type hinting
from openpyxl.cell import MergedCell # To check for merged cells

# --- Configuration ---
# !!! IMPORTANT: Replace this URL with the actual direct download link for the UFGS zip file !!!
# Find this link on https://www.wbdg.org/dod/ufgs - look for the latest "SpecsIntact (SEC) in compressed ZIP"
ufgs_download_url = "https://www.wbdg.org/FFC/DOD/UFGS/UFGS_M.zip" #<-- ### CONFIRM OR UPDATE THIS URL ###

# Define paths for zip file download, extraction, and output Excel file
zip_file_path = '/content/UFGS.zip'  # Path where the zip file will be downloaded
extract_path = '/content/sec_files'
output_excel_path = '/content/SEC_URL_references.xlsx # Changed output name

# Debug mode toggle
debug = True  # Set to True to turn on debug output

# List of specifications and their discipline
TARGET_SPECS_DATA = [
    ("02 84 16", "Electrical"), ("02 84 33", "Electrical"), ("08 34 49.00 20", "Electrical"),
    ("08 71 63.10", "Electrical"), ("26 05 13.00 10", "Electrical"), ("26 05 19.00 10", "Electrical"),
    ("26 05 33", "Electrical"), ("26 05 48", "Electrical"), ("26 05 73", "Electrical"),
    ("26 08 00", "Electrical"), ("26 11 13.00 20", "Electrical"), ("26 11 14.00 10", "Electrical"),
    ("26 11 16", "Electrical"), ("26 12 19", "Electrical"), ("26 12 21", "Electrical"),
    ("26 13 00", "Electrical"), ("26 13 01", "Electrical"), ("26 13 02", "Electrical"),
    ("26 13 13", "Electrical"), ("26 13 14", "Electrical"), ("26 13 32", "Electrical"),
    ("26 19 13", "Electrical"), ("26 20 00", "Electrical"), ("26 22 00.00 10", "Electrical"),
    ("26 23 00", "Electrical"), ("26 24 13", "Electrical"), ("26 27 29", "Electrical"),
    ("26 28 00.00 10", "Electrical"), ("26 29 01.00 10", "Electrical"), ("26 29 02.00 10", "Electrical"),
    ("26 29 23", "Electrical"), ("26 31 00", "Electrical"), ("26 32 15", "Electrical"),
    ("26 33 00", "Electrical"), ("26 33 53", "Electrical"), ("26 35 43", "Electrical"),
    ("26 35 44", "Electrical"), ("26 36 23", "Electrical"), ("26 41 00", "Electrical"),
    ("26 42 13", "Electrical"), ("26 42 15", "Electrical"), ("26 42 17", "Electrical"),
    ("26 42 19.00 10", "Electrical"), ("26 51 00", "Electrical"), ("26 55 53", "Electrical"),
    ("26 56 00", "Electrical"), ("26 56 20", "Electrical"), ("27 05 13.43", "Electrical"),
    ("27 05 26", "Electrical"), ("27 05 29.00 10", "Electrical"), ("27 10 00", "Electrical"),
    ("27 41 00", "Electrical"), ("27 51 16", "Electrical"), ("27 51 23", "Electrical"),
    ("27 53 19", "Electrical"), ("28 08 10", "Electrical"), ("28 10 05", "Electrical"),
    ("28 20 02", "Electrical"), ("33 71 01", "Electrical"), ("33 71 02", "Electrical"),
    ("33 82 00", "Electrical"), ("34 60 13", "Electrical"), ("35 20 20", "Electrical"),
    ("48 14 00", "Electrical"), ("48 15 00", "Electrical"), ("48 16 00", "Electrical")
    # Add more specs and disciplines here in the future
]


# Regular expression to match <URL> tags with multi-line support
url_pattern = re.compile(r'<URL(?:\s+HREF="([^"]*)")?[^>]*>(.*?)</URL>', re.IGNORECASE | re.DOTALL)
# Updated Regex for Metadata based on SpecsIntact Tags
scn_pattern = re.compile(r'<SCN>(.*?)</SCN>', re.IGNORECASE | re.DOTALL) # Section Number
stl_pattern = re.compile(r'<STL>(.*?)</STL>', re.IGNORECASE | re.DOTALL) # Section Title
dte_pattern = re.compile(r'<DTE>(.*?)</DTE>', re.IGNORECASE | re.DOTALL) # Date
pra_pattern = re.compile(r'<PRA>(.*?)</PRA>', re.IGNORECASE | re.DOTALL) # Preparing Authority

# Dictionary to keep track of errors per file (used during URL validation)
error_counts = {}

global_id_counter = 1
# --- End Configuration ---

# --- Functions ---
def debug_print(message):
    """Prints debug messages if debug mode is enabled."""
    if debug:
        print(message)

def is_email(url):
    """Checks if a URL is an email address (mailto: or contains @)."""
    if not isinstance(url, str):
        return False
    return url.startswith("mailto:") or '@' in url

def is_wbdg_url(url):
    """Checks if a URL belongs to the wbdg.org domain."""
    if isinstance(url, str):
        return "wbdg.org" in url.lower()
    return False

def validate_url(url, local_checked_urls):
    """
    Validates a single URL. Checks for email format first, then attempts
    an HTTP request if it's not an email. Returns status and details.
    local_checked_urls is a dictionary local to the calling thread.
    """
    if not isinstance(url, str):
        url = str(url)
    if url in local_checked_urls:
        return local_checked_urls[url]

    if is_email(url):
        result = ("EMAIL", None, None, None, False)
        local_checked_urls[url] = result
        return result

    sanitized_url = re.sub(r'[^\x20-\x7E]+', '', url).strip()
    if not sanitized_url:
        result = ("INVALID", None, None, "Invalid URL format or empty after sanitization", False)
        local_checked_urls[url] = result
        return result

    wbdg_check = is_wbdg_url(sanitized_url)
    error_message, final_url, status_code = None, None, None
    validation_status = "FAIL"

    if not sanitized_url.startswith(("http://", "https://", "ftp://")):
        schemed_url = "http://" + sanitized_url
    else:
        schemed_url = sanitized_url

    try:
        with httpx.Client(http2=True, verify=False, headers={"User-Agent": "Mozilla/5.0"}, follow_redirects=True, timeout=20) as client:
            response = client.get(schemed_url)
            response.raise_for_status()
            final_url = str(response.url)
            status_code = response.status_code
            validation_status = "PASS"
    except httpx.HTTPStatusError as e:
        status_code = e.response.status_code
        error_message = f"HTTP Error: {status_code} {e.response.reason_phrase}"
        final_url = str(e.request.url)
        debug_print(f"HTTP Error for {schemed_url}: {error_message}")
        if url not in error_counts: error_counts[url] = 0
        error_counts[url] += 1
    except httpx.RequestError as e:
        error_message = f"Request Error: {type(e).__name__}"
        final_url = str(e.request.url) if hasattr(e, 'request') and e.request else schemed_url
        debug_print(f"Request Error for {schemed_url}: {error_message} - {e}")
        if url not in error_counts: error_counts[url] = 0
        error_counts[url] += 1
    except Exception as e:
        error_message = f"Unexpected Error: {type(e).__name__} - {e}"
        final_url = schemed_url
        debug_print(f"Unexpected validation error for {schemed_url}: {error_message}")
        if url not in error_counts: error_counts[url] = 0
        error_counts[url] += 1

    result = (validation_status, status_code, final_url, error_message, wbdg_check)
    local_checked_urls[url] = result
    return result

def parse_sec_file_for_urls(file_path, current_global_id):
    """
    Parses a single .SEC file ONLY FOR URLs.
    Implements logic to determine the primary URL based on HREF and TEXT content.
    """
    debug_print(f"  Parsing for URLs: {os.path.basename(file_path)}")
    extracted_rows = []
    urls_found_in_file = 0
    try:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()
        for match in url_pattern.finditer(content):
            url_href = match.group(1) if match.group(1) else None
            url_txt = match.group(2).strip() if match.group(2) else ''

            primary_url = None
            if url_href:
                primary_url = url_href
            elif url_txt and '.' in url_txt:
                primary_url = url_txt

            if primary_url is None:
                 continue

            char_offset = match.start()
            line_number = content.count('\n', 0, char_offset) + 1

            extracted_rows.append((
                current_global_id, os.path.basename(file_path), line_number,
                char_offset, url_txt, url_href, primary_url
            ))
            current_global_id += 1
            urls_found_in_file += 1
    except Exception as e:
        debug_print(f"  Error parsing URLs in {os.path.basename(file_path)}: {e}")

    debug_print(f"  Finished parsing URLs for {os.path.basename(file_path)}. Found {urls_found_in_file} URLs.")
    return extracted_rows, current_global_id

def extract_file_metadata(file_path):
    """
    Opens a .SEC file, reads the beginning, and extracts metadata using regex
    based on common SpecsIntact tags (<SCN>, <STL>, <DTE>, <PRA>).
    Returns a dictionary with the extracted metadata.
    """
    filename = os.path.basename(file_path)
    metadata = {
        'Filename': filename,
        'Section Number': "Not Found",
        'Section Title': "Not Found",
        'Date': "Not Found",
        'Preparing Authority': "Not Found"
    }
    try:
        max_chars_to_read = 8192
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            content_start = file.read(max_chars_to_read)

        scn_match = scn_pattern.search(content_start)
        if scn_match: metadata['Section Number'] = scn_match.group(1).strip()
        stl_match = stl_pattern.search(content_start)
        if stl_match: metadata['Section Title'] = ' '.join(stl_match.group(1).split()).strip()
        dte_match = dte_pattern.search(content_start)
        if dte_match: metadata['Date'] = dte_match.group(1).strip()
        pra_match = pra_pattern.search(content_start)
        if pra_match: metadata['Preparing Authority'] = ' '.join(pra_match.group(1).split()).strip()
    except Exception as e:
        debug_print(f"  Error extracting metadata from {filename}: {e}")
    return metadata

def auto_fit_columns(worksheet: Worksheet, columns_to_fit=None, max_width=70, padding=3):
    """
    Adjusts column widths on an openpyxl worksheet based on content.
    Iterates through specified columns (or all if None) and handles merged cells.
    """
    if columns_to_fit:
        col_indices = [ord(c.upper()) - ord('A') + 1 for c in columns_to_fit]
    else:
        col_indices = range(1, worksheet.max_column + 1)

    for col_idx in col_indices:
        max_length = 0
        column_letter = get_column_letter(col_idx)

        for row_idx in range(1, worksheet.max_row + 1):
            cell = worksheet.cell(row=row_idx, column=col_idx)
            is_merged = isinstance(cell, MergedCell)

            if cell.row == 1 and cell.value:
                 try: max_length = max(max_length, len(str(cell.value)))
                 except: pass
            if is_merged and cell.row > 1: continue
            if not is_merged and cell.value is not None:
                try:
                    cell_length = len(str(cell.value))
                    if cell_length > max_length: max_length = cell_length
                except Exception: pass

        adjusted_width = max_length + padding
        final_width = min(adjusted_width, max_width)
        final_width = max(final_width, 8)
        worksheet.column_dimensions[column_letter].width = final_width

    cols_fitted_str = ", ".join(columns_to_fit) if columns_to_fit else "All"
    debug_print(f"Auto-fitted columns ({cols_fitted_str}) for sheet: {worksheet.title}")
# --- End Helper Function ---


# --- Main Execution ---
start_time = time.time()

# --- Download UFGS Zip File ---
if not ufgs_download_url or "REPLACE_WITH_CORRECT_UFGS_ZIP_DOWNLOAD_LINK" in ufgs_download_url:
    print("Error: The 'ufgs_download_url' variable has not been updated or is still a placeholder.")
    sys.exit(1)
debug_print(f"Attempting to download UFGS zip file from: {ufgs_download_url}")
try:
    with httpx.stream("GET", ufgs_download_url, follow_redirects=True, timeout=60.0, verify=False) as response:
        response.raise_for_status()
        print(f"Downloading to {zip_file_path}...")
        with open(zip_file_path, 'wb') as f:
            for chunk in response.iter_bytes(chunk_size=8192):
                f.write(chunk)
        print(f"\nDownload complete. File saved to {zip_file_path}")
except httpx.HTTPStatusError as e:
    print(f"\nError downloading file: HTTP Status {e.response.status_code} for URL {e.request.url}\nFailed URL: {ufgs_download_url}")
    sys.exit(1)
except httpx.RequestError as e:
    print(f"\nError downloading file: {e}\nFailed URL: {ufgs_download_url}")
    sys.exit(1)
except Exception as e:
    print(f"\nAn unexpected error occurred during download: {e}\nFailed URL: {ufgs_download_url}")
    sys.exit(1)
if not os.path.exists(zip_file_path):
    print(f"Error: Zip file not found at {zip_file_path} after download attempt. Exiting.")
    sys.exit(1)
# --- End Download ---

# --- Extract Zip File ---
if os.path.exists(extract_path):
    shutil.rmtree(extract_path)
os.makedirs(extract_path)
debug_print("Extracting zip file...")
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    debug_print("Extraction complete.")
except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {zip_file_path} is not a valid zip file or is corrupted.")
    sys.exit(1)
except Exception as e:
    print(f"An error occurred during zip extraction: {e}")
    sys.exit(1)
# --- End Extraction ---

# --- Process SEC Files (Extract Metadata and URLs) ---
all_url_rows = []
sec_files_found = []
file_metadata_list = []

for root, dirs, files in os.walk(extract_path):
    for item in files:
        if item.lower().endswith('.sec'):
            full_path = os.path.join(root, item)
            sec_files_found.append(full_path)

if not sec_files_found:
    print(f"Error: No '.SEC' files found in the extracted content at {extract_path}. Exiting.")
    sys.exit(1)

print(f"Found {len(sec_files_found)} .SEC files to process.")
sec_files_found.sort()

for file_path in sec_files_found:
    debug_print(f"-----------------------")
    debug_print(f"Processing file: {os.path.basename(file_path)}")
    metadata = extract_file_metadata(file_path)
    file_metadata_list.append(metadata)
    debug_print(f"  Extracted Metadata: SCN='{metadata['Section Number']}', STL='{metadata['Section Title']}', DTE='{metadata['Date']}', PRA='{metadata['Preparing Authority']}'")
    url_rows, global_id_counter = parse_sec_file_for_urls(file_path, global_id_counter)
    all_url_rows.extend(url_rows)

# --- Create DataFrames ---
df_file_metadata = pd.DataFrame(file_metadata_list)
debug_print(f"Created DataFrame with metadata for {len(df_file_metadata)} files.")

initial_url_columns = ['ID', 'SEC_FILE_NAME', 'LINE_NUMBER', 'CHAR_OFFSET', 'URL_TXT', 'URL_HREF', 'PRIMARY_URL']
if not all_url_rows:
     print("Warning: No URLs found in any of the processed .SEC files.")
     df_urls = pd.DataFrame(columns=initial_url_columns)
else:
    df_urls = pd.DataFrame(all_url_rows, columns=initial_url_columns)
debug_print(f"Created DataFrame with {len(df_urls)} total URL entries.")

# --- Add Discipline Column using Pandas Merge ---
df_specs = pd.DataFrame(TARGET_SPECS_DATA, columns=['UFGS', 'Discipline_Spec'])
def get_base_spec(filename):
    if isinstance(filename, str) and filename.lower().endswith('.sec'):
        return filename[:-4]
    return filename
df_urls['Base_Spec'] = df_urls['SEC_FILE_NAME'].apply(get_base_spec)
df_urls = pd.merge(df_urls, df_specs, left_on='Base_Spec', right_on='UFGS', how='left')
df_urls.rename(columns={'Discipline_Spec': 'Discipline'}, inplace=True)
df_urls['Discipline'] = df_urls['Discipline'].fillna('Other')
df_urls.drop(columns=['Base_Spec', 'UFGS'], inplace=True)
debug_print("Added 'Discipline' column to URL DataFrame.")
# --- End Add Discipline ---


# --- URL Validation ---
unique_primary_urls = df_urls['PRIMARY_URL'].dropna().unique().tolist()
print(f"Validating {len(unique_primary_urls)} unique URLs...")
validation_results_map = {}
url_validation_start_time = time.time()
MAX_WORKERS = 10
urls_processed = 0
if unique_primary_urls:
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_url = {executor.submit(validate_url, url, {}): url for url in unique_primary_urls}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                validation_results_map[url] = result
                urls_processed += 1
                print(f"Validated {urls_processed}/{len(unique_primary_urls)} URLs...", end='\r')
            except Exception as exc:
                print(f'\nURL {url} generated an exception during validation task processing: {exc}')
                validation_results_map[url] = ("ERROR", None, None, str(exc), is_wbdg_url(url))
    print(f"Validated {urls_processed}/{len(unique_primary_urls)} URLs.          ")
    print(f"URL Validation finished in {time.time() - url_validation_start_time:.2f} seconds.")
else:
    print("Skipping URL validation as no URLs were found.")

# Map validation results back
result_columns = ['STATUS', 'RESPONSE_CODE', 'FINAL_URL', 'ERROR_MSG', 'IS_WBDG']
if not df_urls.empty:
    validation_series = df_urls['PRIMARY_URL'].map(validation_results_map)
    def get_result_part(series_item, index):
        if isinstance(series_item, tuple) and len(series_item) > index:
            return series_item[index]
        return None
    for i, col_name in enumerate(result_columns):
        df_urls[col_name] = validation_series.apply(lambda x: get_result_part(x, i))
    df_urls['COUNT'] = df_urls.groupby('PRIMARY_URL')['PRIMARY_URL'].transform('count')
else:
    for col in result_columns + ['COUNT']:
        df_urls[col] = None
# --- End URL Validation ---

# --- Calculate Summary Statistics ---
debug_print("Calculating summary statistics...")
summary_data = {}
if not df_urls.empty:
    total_urls = len(df_urls); unique_urls = df_urls['PRIMARY_URL'].nunique()
    error_df = df_urls[df_urls['STATUS'] == 'FAIL']
    total_errors = len(error_df); unique_errors = error_df['PRIMARY_URL'].nunique()
    summary_data['Overall'] = {
        'Total URLs Processed': total_urls, 'Total URL Errors (FAIL)': total_errors,
        'Unique URLs Processed': unique_urls, 'Unique URL Errors (FAIL)': unique_errors,
        '% Total URLs with Errors': None, '% Unique URLs with Errors': None
    }
    discipline_summary = {}
    grouped = df_urls.groupby('Discipline')
    for name, group in grouped:
        disc_total_urls = len(group); disc_unique_urls = group['PRIMARY_URL'].nunique()
        disc_error_df = group[group['STATUS'] == 'FAIL']
        disc_total_errors = len(disc_error_df); disc_unique_errors = disc_error_df['PRIMARY_URL'].nunique()
        discipline_summary[name] = {
            'Total URLs': disc_total_urls, 'Total Errors (FAIL)': disc_total_errors,
            'Unique URLs': disc_unique_urls, 'Unique Errors (FAIL)': disc_unique_errors,
            '% Total URLs Error': None, '% Unique URLs Error': None
        }
    summary_data['By Discipline'] = dict(sorted(discipline_summary.items(), key=lambda item: (item[0] == 'Other', item[0])))
else:
    summary_data['Overall'] = {
        'Total URLs Processed': 0, 'Total URL Errors (FAIL)': 0,
        'Unique URLs Processed': 0, 'Unique URL Errors (FAIL)': 0,
        '% Total URLs with Errors': None, '% Unique URLs with Errors': None
    }
    summary_data['By Discipline'] = {}
debug_print("Summary statistics calculated (counts only).")
# --- End Calculate Summary Statistics ---


# --- Save to Excel ---
debug_print(f"Saving DataFrame to Excel: {output_excel_path}")
try:
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:

        # --- Define Sheet Names and Columns ---
        summary_sheet_name = 'Summary'
        main_sheet_name = 'URL_Checks'
        file_list_sheet_name = 'File_List'
        spec_list_sheet_name = 'Spec_List'

        output_columns_main = ['ID', 'SEC_FILE_NAME', 'Discipline', 'STATUS', 'COUNT', 'IS_WBDG','LINE_NUMBER', 'CHAR_OFFSET', 'URL_TXT', 'PRIMARY_URL','RESPONSE_CODE', 'FINAL_URL', 'ERROR_MSG']
        output_columns_filelist = ['Filename', 'Section Number', 'Section Title', 'Date', 'Preparing Authority']

        # --- Create and Write Summary Sheet ---
        ws_summary = writer.book.create_sheet(summary_sheet_name, 0)
        debug_print(f"Created summary sheet: {summary_sheet_name}")
        # Define styles
        header_font = Font(bold=False, size=24) # Size 24, Not Bold
        sub_header_font = Font(bold=True)
        percent_format_string = '0.0%'; header_fill = PatternFill(start_color="DDEEFF", end_color="DDEEFF", fill_type="solid")
        grey_fill = PatternFill(start_color="EEEEEE", end_color="EEEEEE", fill_type="solid")
        thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
        center_align = Alignment(horizontal='center', vertical='center')

        current_row = 1
        # Overall Summary Header
        overall_header_cell = ws_summary.cell(row=current_row, column=1, value="Overall Summary"); overall_header_cell.font = header_font; overall_header_cell.fill = header_fill
        ws_summary.merge_cells(start_row=current_row, start_column=1, end_row=current_row, end_column=2); current_row += 1
        overall_cells = {}
        # Write Overall Data and apply borders/alignment
        overall_start_row = current_row
        overall_keys_ordered = ['Total URLs Processed', 'Total URL Errors (FAIL)','Unique URLs Processed', 'Unique URL Errors (FAIL)','% Total URLs with Errors', '% Unique URLs with Errors']
        for key in overall_keys_ordered:
            value = summary_data['Overall'].get(key)
            label_cell = ws_summary.cell(row=current_row, column=1, value=key)
            value_cell = ws_summary.cell(row=current_row, column=2, value=value)
            overall_cells[key] = value_cell.coordinate
            value_cell.alignment = center_align
            current_row += 1
        # Write Overall Formulas and apply format/alignment
        if summary_data['Overall']['Total URLs Processed'] > 0: formula_total_err_pct = f'=IFERROR({overall_cells["Total URL Errors (FAIL)"]}/{overall_cells["Total URLs Processed"]},0)'; cell_total_err_pct = ws_summary[overall_cells['% Total URLs with Errors']]; cell_total_err_pct.value = formula_total_err_pct; cell_total_err_pct.number_format = percent_format_string; cell_total_err_pct.alignment = center_align
        else: ws_summary[overall_cells['% Total URLs with Errors']].value = 0; ws_summary[overall_cells['% Total URLs with Errors']].number_format = percent_format_string; ws_summary[overall_cells['% Total URLs with Errors']].alignment = center_align
        if summary_data['Overall']['Unique URLs Processed'] > 0: formula_unique_err_pct = f'=IFERROR({overall_cells["Unique URL Errors (FAIL)"]}/{overall_cells["Unique URLs Processed"]},0)'; cell_unique_err_pct = ws_summary[overall_cells['% Unique URLs with Errors']]; cell_unique_err_pct.value = formula_unique_err_pct; cell_unique_err_pct.number_format = percent_format_string; cell_unique_err_pct.alignment = center_align
        else: ws_summary[overall_cells['% Unique URLs with Errors']].value = 0; ws_summary[overall_cells['% Unique URLs with Errors']].number_format = percent_format_string; ws_summary[overall_cells['% Unique URLs with Errors']].alignment = center_align
        overall_end_row = current_row -1

        # Discipline Summary Header
        current_row += 1
        discipline_header_row = current_row
        discipline_header_cell = ws_summary.cell(row=current_row, column=1, value="Summary by Discipline"); discipline_header_cell.font = header_font; discipline_header_cell.fill = header_fill
        ws_summary.merge_cells(start_row=current_row, start_column=1, end_row=current_row, end_column=7);
        for col_idx in range(1, 8): ws_summary.cell(row=current_row, column=col_idx).fill = header_fill
        current_row += 1
        # Discipline Summary Data & Formulas
        discipline_start_row = current_row
        if summary_data['By Discipline']:
            disc_headers_ordered = ['Total URLs', 'Total Errors (FAIL)','Unique URLs', 'Unique Errors (FAIL)','% Total URLs Error', '% Unique URLs Error']
            # Write Discipline Headers and apply formatting
            discipline_col_header_cell = ws_summary.cell(row=current_row, column=1, value="Discipline"); discipline_col_header_cell.font = sub_header_font; discipline_col_header_cell.fill = grey_fill; discipline_col_header_cell.border = thin_border
            for col_idx, header in enumerate(disc_headers_ordered): cell = ws_summary.cell(row=current_row, column=col_idx + 2, value=header); cell.font = sub_header_font; cell.fill = grey_fill; cell.border = thin_border; cell.alignment = center_align
            header_row_num = current_row; current_row += 1
            # Write Discipline Data/Formulas and apply formatting
            for discipline, stats in summary_data['By Discipline'].items():
                disc_label_cell = ws_summary.cell(row=current_row, column=1, value=discipline); disc_label_cell.font = sub_header_font; disc_label_cell.border = thin_border
                discipline_cells = {}; col_idx = 2
                for key in disc_headers_ordered:
                    value = stats.get(key)
                    cell = ws_summary.cell(row=current_row, column=col_idx, value=value); discipline_cells[key] = cell.coordinate; cell.border = thin_border; cell.alignment = center_align; col_idx += 1
                # Formulas
                if stats['Total URLs'] > 0: formula_disc_total_err_pct = f'=IFERROR({discipline_cells["Total Errors (FAIL)"]}/{discipline_cells["Total URLs"]},0)'; cell_disc_total_err_pct = ws_summary[discipline_cells['% Total URLs Error']]; cell_disc_total_err_pct.value = formula_disc_total_err_pct; cell_disc_total_err_pct.number_format = percent_format_string; cell_disc_total_err_pct.alignment = center_align; cell_disc_total_err_pct.border = thin_border
                else: ws_summary[discipline_cells['% Total URLs Error']].value = 0; ws_summary[discipline_cells['% Total URLs Error']].number_format = percent_format_string; ws_summary[discipline_cells['% Total URLs Error']].alignment = center_align; ws_summary[discipline_cells['% Total URLs Error']].border = thin_border
                if stats['Unique URLs'] > 0: formula_disc_unique_err_pct = f'=IFERROR({discipline_cells["Unique Errors (FAIL)"]}/{discipline_cells["Unique URLs"]},0)'; cell_disc_unique_err_pct = ws_summary[discipline_cells['% Unique URLs Error']]; cell_disc_unique_err_pct.value = formula_disc_unique_err_pct; cell_disc_unique_err_pct.number_format = percent_format_string; cell_disc_unique_err_pct.alignment = center_align; cell_disc_unique_err_pct.border = thin_border
                else: ws_summary[discipline_cells['% Unique URLs Error']].value = 0; ws_summary[discipline_cells['% Unique URLs Error']].number_format = percent_format_string; ws_summary[discipline_cells['% Unique URLs Error']].alignment = center_align; ws_summary[discipline_cells['% Unique URLs Error']].border = thin_border
                current_row += 1
        else: ws_summary.cell(row=current_row, column=1, value="No discipline data found."); current_row +=1
        discipline_end_row = current_row - 1

        # Apply Borders to Summary Sections
        for r in range(overall_start_row, overall_end_row + 1):
            for c in range(1, 3): ws_summary.cell(row=r, column=c).border = thin_border
        if summary_data['By Discipline']:
             for r in range(discipline_start_row, discipline_end_row + 1):
                 for c in range(1, 8): ws_summary.cell(row=r, column=c).border = thin_border
        debug_print(f"Formatted summary sheet: {summary_sheet_name}")
        # --- End Summary Sheet ---


        # --- Write Main Sheet ('URL_Checks') ---
        if not df_urls.empty:
             for col in output_columns_main:
                 if col not in df_urls.columns: df_urls[col] = ''
             df_to_save = df_urls[output_columns_main].copy()
        else:
             df_to_save = pd.DataFrame(columns=output_columns_main)
        df_to_save = df_to_save.fillna('')
        df_to_save.to_excel(writer, index=False, sheet_name=main_sheet_name)
        ws_main = writer.sheets[main_sheet_name]
        debug_print(f"Written main sheet: {main_sheet_name}")
        # --- End Write Main Sheet ---


        # --- Write File List Sheet ---
        ws_file_list = writer.book.create_sheet(file_list_sheet_name)
        debug_print(f"Created file list sheet: {file_list_sheet_name}")
        df_file_metadata_to_save = df_file_metadata[output_columns_filelist].sort_values(by='Filename').reset_index(drop=True)
        df_file_metadata_to_save.to_excel(writer, index=False, sheet_name=file_list_sheet_name)
        debug_print(f"Written file metadata to sheet: {file_list_sheet_name}")
        ws_file_list.freeze_panes = "A2"
        if not df_file_metadata_to_save.empty:
             from openpyxl.worksheet.table import Table, TableStyleInfo
             table_files = Table(displayName="FileListTable", ref=f"A1:E{len(df_file_metadata_to_save)+1}")
             style_files = TableStyleInfo(name="TableStyleMedium2", showFirstColumn=False, showLastColumn=False, showRowStripes=True, showColumnStripes=False)
             table_files.tableStyleInfo = style_files
             if not any(t.name == "FileListTable" for t in ws_file_list._tables): ws_file_list.add_table(table_files)
             # Format File List Header (White Font, Not Bold)
             white_font = Font(color="FFFFFF", bold=False)
             for col_idx in range(1, len(output_columns_filelist) + 1):
                 header_cell = ws_file_list.cell(row=1, column=col_idx)
                 header_cell.font = white_font
        debug_print(f"Formatted file list sheet: {file_list_sheet_name}")
        # --- End File List Sheet ---


        # --- Create and Populate Spec List Sheet ---
        ws_spec_list = writer.book.create_sheet(spec_list_sheet_name)
        debug_print(f"Created spec list sheet: {spec_list_sheet_name}")
        ws_spec_list['A1'] = "UFGS"; ws_spec_list['B1'] = "Discipline"
        spec_list_start_row = 2
        for i, (spec, discipline) in enumerate(TARGET_SPECS_DATA):
            ws_spec_list.cell(row=spec_list_start_row + i, column=1, value=spec)
            ws_spec_list.cell(row=spec_list_start_row + i, column=2, value=discipline)
        debug_print(f"Written {len(TARGET_SPECS_DATA)} target specs and disciplines to {spec_list_sheet_name}")
        ws_spec_list.freeze_panes = "A2"
        if len(TARGET_SPECS_DATA) > 0:
             from openpyxl.worksheet.table import Table, TableStyleInfo
             table_spec = Table(displayName="SpecListTable", ref=f"A1:B{len(TARGET_SPECS_DATA)+1}")
             style_spec = TableStyleInfo(name="TableStyleMedium2", showFirstColumn=False,showLastColumn=False, showRowStripes=True, showColumnStripes=False)
             table_spec.tableStyleInfo = style_spec
             if not any(t.name == "SpecListTable" for t in ws_spec_list._tables): ws_spec_list.add_table(table_spec)
             # Format Spec List Header (White Font, Not Bold)
             white_font = Font(color="FFFFFF", bold=False)
             for col_idx in range(1, 3): # Columns A and B
                 header_cell = ws_spec_list.cell(row=1, column=col_idx)
                 header_cell.font = white_font
        debug_print(f"Formatted spec list sheet: {spec_list_sheet_name}")
        # --- End Spec List Sheet ---


        # --- Format Main Sheet ('URL_Checks') ---
        red_fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
        red_font = Font(color='9C0006')
        green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
        green_font = Font(color='006100')
        data_rows_count_main = len(df_to_save)
        if data_rows_count_main > 0:
            last_col_letter_main = get_column_letter(len(output_columns_main))
            formatting_range_main = f"A2:{last_col_letter_main}{data_rows_count_main + 1}"
            try:
                status_col_index_main = output_columns_main.index('STATUS')
                status_col_letter_main = get_column_letter(status_col_index_main + 1)
                fail_formula_main = f'${status_col_letter_main}2="FAIL"'
                ws_main.conditional_formatting.add(formatting_range_main, FormulaRule(formula=[fail_formula_main], stopIfTrue=False, font=red_font, fill=red_fill))
                pass_formula_main = f'${status_col_letter_main}2="PASS"'
                ws_main.conditional_formatting.add(formatting_range_main, FormulaRule(formula=[pass_formula_main], stopIfTrue=False, font=green_font, fill=green_fill))
                debug_print(f"Applied conditional formatting to main sheet range: {formatting_range_main}")
            except ValueError: debug_print("Could not find 'STATUS' column for main sheet conditional formatting.")
        else: debug_print("Skipping main sheet conditional formatting (no data rows).")
        if len(output_columns_main) > 0 :
            from openpyxl.worksheet.table import Table, TableStyleInfo
            table_range_end_row_main = max(1, len(df_to_save)) + 1
            table_range_main = f"A1:{get_column_letter(len(output_columns_main))}{table_range_end_row_main}"
            table_main = Table(displayName="URLCheckTable", ref=table_range_main)
            # --- Apply Table Style Light 9 ---
            style_main = TableStyleInfo(name="TableStyleLight9", showFirstColumn=False, showLastColumn=False, showRowStripes=True, showColumnStripes=False)
            table_main.tableStyleInfo = style_main
            if not any(t.name == "URLCheckTable" for t in ws_main._tables): ws_main.add_table(table_main)
            # Format Main Sheet Header (White Font, Not Bold)
            white_font = Font(color="FFFFFF", bold=False)
            for col_idx in range(1, len(output_columns_main) + 1):
                header_cell = ws_main.cell(row=1, column=col_idx)
                header_cell.font = white_font
        ws_main.freeze_panes = "A2"
        debug_print(f"Formatted main sheet: {main_sheet_name}")
        # --- End Main Sheet Formatting ---

        # --- Apply Auto-Fit to Specific Columns/Sheets ---
        auto_fit_columns(ws_summary, columns_to_fit=['A', 'B', 'C', 'D', 'E', 'F', 'G']) # Fit A-G for Summary
        auto_fit_columns(ws_main) # Fit all for URL_Checks
        auto_fit_columns(ws_file_list) # Fit all for File_List
        auto_fit_columns(ws_spec_list) # Fit all for Spec_List
        # --- End Auto-Fit ---

        # --- Ensure correct sheet order ---
        desired_order = [summary_sheet_name, main_sheet_name, file_list_sheet_name, spec_list_sheet_name]
        current_sheets = writer.book.sheetnames
        current_indices = {name: i for i, name in enumerate(current_sheets)}
        sorted_sheets = sorted(current_sheets, key=lambda name: desired_order.index(name) if name in desired_order else len(desired_order))
        writer.book._sheets = [writer.book[name] for name in sorted_sheets]
        debug_print(f"Set sheet order to: {', '.join(writer.book.sheetnames)}")


    # End of ExcelWriter context manager, file is saved here
    debug_print(f"Excel file successfully created at: {output_excel_path}")

except Exception as e:
    # Catch errors during Excel writing/formatting
    print(f"\nError saving DataFrame to Excel: {e}")
    import traceback
    traceback.print_exc() # Print detailed traceback for Excel errors

# --- Final Summary ---
end_time = time.time()
total_time = end_time - start_time
print(f"\nTotal Processing Time: {time.strftime('%H:%M:%S', time.gmtime(total_time))}")
print("\nURL Validation Error Summary (by Primary URL):")
if error_counts:
    sorted_errors = sorted(error_counts.items(), key=lambda item: item[1], reverse=True)
    for url, count in sorted_errors:
        error_msg = validation_results_map.get(url, (None, None, None, "Unknown error"))[3]
        print(f"  - Count: {count}, URL: {url[:80]}... , Last Error: {error_msg}")
else:
    if unique_primary_urls:
         print("  No URL validation errors recorded.")
    else:
         print("  URL validation skipped (no URLs found).")
print(f"\nAnalysis complete. Output saved to {output_excel_path}")
print(f"Includes sheet '{summary_sheet_name}' with overall and discipline summaries.")
print(f"Includes sheet '{file_list_sheet_name}' with SEC file details (Filename, SCN, STL, DTE, PRA).")
print(f"Includes sheet '{spec_list_sheet_name}' with target specs and disciplines.")

Attempting to download UFGS zip file from: https://www.wbdg.org/FFC/DOD/UFGS/UFGS_M.zip

Download complete. File saved to /content/UFGS.zip
Extracting zip file...
Extraction complete.
Found 696 .SEC files to process.
-----------------------
Processing file: 00 01 15.SEC
  Extracted Metadata: SCN='DOCUMENT 00 01 15', STL='LIST OF DRAWINGS', DTE='02/24', PRA='NAVFAC'
  Parsing for URLs: 00 01 15.SEC
  Finished parsing URLs for 00 01 15.SEC. Found 2 URLs.
-----------------------
Processing file: 01 11 00.SEC
  Extracted Metadata: SCN='SECTION 01 11 00', STL='SUMMARY OF WORK', DTE='02/24', PRA='NAVFAC'
  Parsing for URLs: 01 11 00.SEC
  Finished parsing URLs for 01 11 00.SEC. Found 2 URLs.
-----------------------
Processing file: 01 14 00.SEC
  Extracted Metadata: SCN='SECTION 01 14 00', STL='WORK RESTRICTIONS', DTE='11/22, CHG 4: 02/25', PRA='NAVFAC'
  Parsing for URLs: 01 14 00.SEC
  Finished parsing URLs for 01 14 00.SEC. Found 13 URLs.
-----------------------
Processing file: 01 20 00.